In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load in Data from GitHub
data = pd.read_csv('final_data.csv')
data.head()

In [ ]:
data.columns

In [ ]:
# Combine Director and Writer a single column with comma-separated values
data['Team'] = data.apply(lambda row: ', '.join(filter(None, [str(row['director']), str(row['writer'])])), axis=1)

In [ ]:
data.drop(columns=['audience_score', 'tomato_meter','wiki_page', 'director', 'writer'], inplace=True)
data.head()

In [ ]:
data['cast'] = (
    data['cast']
    .str.replace(";", ",", regex=False)
    .str.replace("and", ",", regex=False)
    .str.replace("with", ",", regex=False)
    .str.replace("voices of ", ",", regex=False)
    .str.replace("\r\n", ",", regex=False)
    .str.replace(r"\(.*?\)", "", regex=True)  # Remove text within parentheses
    .str.replace("Cast:", "", regex=False)        # Remove "Cast:"
)

data['Team'] = (
    data['Team']
    .str.replace(";", ",", regex=False)
    .str.replace("and", ",", regex=False)
    .str.replace("with", ",", regex=False)
    .str.replace("voices of ", ",", regex=False)
    .str.replace("\r\n", ",", regex=False)
    .str.replace(r"\(.*?\)", "", regex=True)  # Remove text within parentheses
    .str.replace("Director :", "", regex=False)  # Remove "Director :"
)
data.head()

In [ ]:
data['Team'] = data['Team'].fillna('').str.split(r',\s*')
data['cast'] = data['cast'].fillna('').str.split(r',\s*')
data.head()

## Try Latent Factor Extraction

In [ ]:
# Import MultiLabelBinarizer
#from sklearn.preprocessing import MultiLabelBinarizer

# Create a one-hot encoded matrix from the list of names
#mlb = MultiLabelBinarizer()

#name_matrix = mlb.fit_transform(data2['names'])

#name_df = pd.DataFrame(name_matrix, columns=mlb.classes_)

#actor_counts = name_df.sum(axis=0)

# Filter out actors that appear only once (i.e., count == 1)
#filtered_name_df = name_df.loc[:, actor_counts > 10]

#print("One-hot encoded matrix:")
#print(name_df)


In [ ]:
#fmatrix = filtered_name_df.values

# Alternatively, using .to_numpy()
#fmatrix = filtered_name_df.to_numpy()

In [ ]:
#import matplotlib.pyplot as plt
#from sklearn.decomposition import TruncatedSVD

# Use a higher number of components to inspect explained variance
#n_components_initial = 2500
#svd = TruncatedSVD(n_components=n_components_initial, random_state=42)

#svd.fit_transform(name_matrix)

#explained_variance = svd.explained_variance_ratio_
#cumulative_variance = explained_variance.cumsum()

# Plot the cumulative explained variance (Scree Plot)
#plt.figure(figsize=(8, 4))
#plt.plot(range(1, n_components_initial + 1), cumulative_variance, marker='o')
#plt.xlabel('Number of Components')
#plt.ylabel('Cumulative Explained Variance')
#plt.title('Scree Plot for TSVD Components')
#plt.axhline(y=0.8, color='r', linestyle='--', label='80% Variance Threshold')
#plt.legend()
#plt.show()


In [ ]:
#movies = name_df.apply(lambda row: list(name_df.columns[row == 1]), axis=1).tolist()

#exclusion_tokens = {" ", "Jr.", "on", "er", "ler", '', 'Am', 'S', 'eras', 'a peet'
#}

# 
#   cleaned_movies = [
#    [actor for actor in movie if actor not in exclusion_tokens]
#    for movie in movies
#]


In [ ]:
#from gensim import corpora, models

# Create a dictionary mapping actor names to IDs
#dictionary = corpora.Dictionary(cleaned_movies)

# Convert each movie into a bag-of-actors representation
#corpus = [dictionary.doc2bow(movie) for movie in movies]

In [ ]:
#num_topics = 10 
#lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, random_state=42)

In [ ]:
#import pyLDAvis
#import pyLDAvis.gensim_models as gensimvis

# Enable notebook display for pyLDAvis
#pyLDAvis.enable_notebook()

# Prepare the visualization data
#vis_data = gensimvis.prepare(lda_model, corpus, dictionary)

# Display the visualization inline in your notebook
#vis_data


In [ ]:
#import pandas as pd
#import numpy as np
#from scipy.sparse import csr_matrix
#from sklearn.decomposition import TruncatedSVD
#import matplotlib.pyplot as plt

# Assume name_df is already defined and is a DataFrame with shape (2958, 4823)
# For example, you might have:
# name_df = pd.read_csv('your_file.csv')

# Convert the DataFrame to a SciPy sparse matrix (CSR format)
#sparse_matrix = csr_matrix(name_df.values)

# Define a range of SVD components to test (e.g., from 10 to 200 in steps of 10)
#component_range = range(100, 1000, 100)
#cumulative_explained = []

# Iterate over the range of components and compute cumulative explained variance
#for n_components in component_range:
#    svd = TruncatedSVD(n_components=n_components, random_state=42)
#    svd.fit(sparse_matrix)
#    explained = np.sum(svd.explained_variance_ratio_)
#    cumulative_explained.append(explained)
#   print(f"n_components: {n_components}, Cumulative Explained Variance: {explained:.4f}")

# Plot cumulative explained variance vs. number of components
#plt.figure(figsize=(8, 4))
#plt.plot(list(component_range), cumulative_explained, marker='o')
#plt.xlabel('Number of Components')
#plt.ylabel('Cumulative Explained Variance')
#plt.title('Optimal Number of SVD Components')
#plt.grid(True)
#plt.show()

In [ ]:
#import pandas as pd
#from scipy.sparse import csr_matrix
#from sklearn.decomposition import TruncatedSVD
#from sklearn.cluster import KMeans
#from sklearn.preprocessing import normalize
#from sklearn.metrics import silhouette_score
#import matplotlib.pyplot as plt

#svd = TruncatedSVD(n_components=400, random_state=42)
#movie_features = svd.fit_transform(sparse_matrix)

# Normalize the feature vectors
#movie_features = normalize(movie_features)

# Define a range of cluster numbers to try
#cluster_range = range(2, 25)  # trying cluster counts from 2 to 15
#silhouette_scores = []

#for n_clusters in cluster_range:
#    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
#    cluster_labels = kmeans.fit_predict(movie_features)
#    score = silhouette_score(movie_features, cluster_labels)
#    silhouette_scores.append(score)
#    print(f"Number of clusters: {n_clusters} - Silhouette Score: {score:.4f}")

# Plot silhouette scores to visually inspect the optimal cluster count
#plt.figure(figsize=(8, 4))
#plt.plot(list(cluster_range), silhouette_scores, marker='o')
#plt.title("Silhouette Score vs. Number of Clusters")
#plt.xlabel("Number of Clusters")
#plt.ylabel("Silhouette Score")
#plt.xticks(list(cluster_range))
#plt.grid(True)
#plt.show()

In [ ]:
#sparse_matrix = csr_matrix(name_df.values)

# Define a range of SVD components to test
#component_options = [50, 100, 200, 400,500,600, 700, 800, 900]
#sil_scores = {}

#for n_components in component_options:
#    svd = TruncatedSVD(n_components=n_components, random_state=42)
#    features = svd.fit_transform(sparse_matrix)
#    features = normalize(features)
    
    # Use a fixed number of clusters; you can adjust based on your earlier analysis
#    kmeans = KMeans(n_clusters=10, random_state=42)
    labels = kmeans.fit_predict(features)
    
 #   score = silhouette_score(features, labels)
 #   sil_scores[n_components] = score
 #   print(f"n_components: {n_components} - Silhouette Score: {score:.4f}")

# Plot the silhouette scores against the number of components
#plt.figure(figsize=(8, 4))
#plt.plot(list(sil_scores.keys()), list(sil_scores.values()), marker='o')
#plt.xlabel('Number of SVD Components')
#plt.ylabel('Silhouette Score')
#plt.title('Silhouette Score vs. SVD Components')
#plt.grid(True)
#plt.show()

## Reviews Mapped to Team Members (Cast, Director, Writer) 

In [ ]:
# Load in Review
reviews = pd.read_csv('Rotten/rotten_tomatoes_movie_reviews.csv')
reviews.head()

In [ ]:
# Load in Movies
movies = pd.read_csv('Rotten/rotten_tomatoes_movies.csv')
movies.head()

In [ ]:
# Merge movies and reviews by 'id'
merged_df = pd.merge(movies, reviews, on='id', how='inner')
merged_df.head()

In [ ]:
merged_df.shape

In [ ]:
# Get unique names from the TEAM OR CAST column
#unique_names = pd.Series([name for sublist in data['cast'].apply(lambda x: list(set(x))).tolist() for name in sublist]).unique()
#unique_names = unique_names[unique_names != '']

unique_names = pd.Series([name for sublist in data['Team'].apply(lambda x: list(set(x))).tolist() for name in sublist]).unique()
unique_names = unique_names[unique_names != '']

unique_names.shape

In [ ]:
# Drop duplicates and clean names
unique_names = pd.Series(unique_names).drop_duplicates().str.strip()
sorted_unique_names = sorted([name for name in unique_names if name and len(name.split()) >= 2 and len(name) > 3])
len(sorted_unique_names)

In [ ]:
sorted_unique_names2 = [name for name in sorted_unique_names if len(name.split()[0]) != 1 and len(name.split()[-1]) > 1]
len(sorted_unique_names2)

In [ ]:
final_names = [name for name in sorted_unique_names2 if len(name.split()) < 4]
len(final_names)

In [ ]:
### DONT RUN THIS CELL

# Create a list to store the reviews for each actor
reviews = []

# Compile regex pattern once for efficiency
name_patterns = {name: re.compile(r'\b' + re.escape(name) + r'\b', re.IGNORECASE) for name in final_names}

# Iterate through each review in merged_df
for index, row in merged_df.iterrows():
    print(f"Processing row {index} ({(index / 1469543) * 100:.2f}%)")
    review_text = row['reviewText']
    if pd.isna(review_text):
        continue
    seen_review_ids = set()
    # Check each name pattern against the review text
    for name, pattern in name_patterns.items():
        if pattern.search(review_text) and row['reviewId'] not in seen_review_ids:
            reviews.append({
                'Name': name, 
                'Review': review_text, 
                'reviewId': row['reviewId'], 
                'creationDate': row['creationDate'],
                'audienceScore': row['audienceScore'],
                'tomatoMeter': row['tomatoMeter']
            })
            seen_review_ids.add(row['reviewId'])

# Convert the list to a DataFrame
#actor_reviews_df = pd.DataFrame(reviews)
team_reviews_df = pd.DataFrame(reviews)
#actor_reviews_df.head()
team_reviews_df.head()

In [ ]:
#actor_reviews_df.sort_values('Name')
team_reviews_df.sort_values('Name')

In [ ]:
#actor_reviews_df['Name'].nunique()
team_reviews_df['Name'].nunique()

In [ ]:
# Save the DataFrame to a CSV file
#actor_reviews_df.to_csv('actor_reviews.csv', index=False)
team_reviews_df.to_csv('team_reviews.csv', index=False)

In [ ]:
# Save the DataFrame to a JSON file
#import json

#with open('actor_reviews.json', 'w') as f:
#    json.dump(actor_reviews, f, indent=4)

In [ ]:
# load in Reviews - ran prior in above cells
dat = pd.read_csv('actor_reviews.csv')
#team = pd.read_csv('team_reviews.csv')

In [ ]:
dat1 = dat.drop_duplicates()
#team1 = team.drop_duplicates()

In [ ]:
del dat
#del team

In [ ]:
# Combine dat1 and team1
#combined_dat = pd.concat([dat1, team1], ignore_index=True)

In [ ]:
#combined_dat = combined_dat.drop_duplicates()

In [ ]:
### Calculate Sentiment Scores for each Actor Review
analyzer = SentimentIntensityAnalyzer()

dat1['Sentiment'] = dat1['Review'].apply(lambda review: analyzer.polarity_scores(review))
#team1['Sentiment'] = team1['Review'].apply(lambda review: analyzer.polarity_scores(review))

In [ ]:
# Extract Main Score 
dat1['compound'] = dat1['Sentiment'].apply(lambda sentiment: sentiment['compound'])
dat1 = dat1.sort_values('Name').reset_index(drop=True)

#team1['compound'] = team1['Sentiment'].apply(lambda sentiment: sentiment['compound'])
#team1 = team1.sort_values('Name').reset_index(drop=True)

In [ ]:
dat1['creationDate'] = pd.to_datetime(dat1['creationDate'])

In [ ]:
data.head()

In [ ]:
data['release_year'] = pd.to_datetime(data['release_year'].astype(str) + '-01-01')

In [ ]:
## Expand Data by Cast Members
movie_exploded = data.explode(['cast'])
#movie_exploded = movie_exploded.explode('Team')

In [ ]:
movie_exploded.head()

In [ ]:
# Check Title Volume
movie_exploded['title'].nunique()

In [ ]:
movie_exploded.shape

In [ ]:
del data

In [ ]:
# Merge with Final Data
total = movie_exploded.merge(dat1,left_on='cast', right_on='Name', how='left')

In [ ]:
total.head()

In [ ]:
del movie_exploded
del dat1

In [ ]:
total.rename(columns={
    'audienceScore': 'Cast_audienceScore',
    'tomatoMeter': 'Cast_tomatoMeter',
    'Sentiment': 'Cast_Sentiment',
    'compound': 'Cast_compound',
    'Review': 'Cast_Review',
    'reviewId': 'Cast_reviewId',
    'creationDate': 'Cast_creationDate'
}, inplace=True)
total.head()

In [ ]:
# Check Title Overlap
total['title'].nunique()

In [ ]:
total.shape

In [ ]:
# Drop duplicates in the 'total' DataFrame
total.drop(columns=['Cast_Sentiment'], inplace=True)

In [ ]:
# Drop duplicates in the 'total' DataFrame
total.drop_duplicates(inplace=True)

In [ ]:
total.shape

In [ ]:
# Create Key for Aggregation
total['movie_id'] = total['title'] + '_' + total['release_year'].dt.year.astype(str)
total.head()

In [ ]:
# Group by 'movie_id', aggregate metrics and keep all other columns. Make sure to filter for point in time data
aggregated_data = total.groupby('movie_id').apply(lambda group: pd.Series({
    #'Team_Review_Score_sum': group.loc[group['Team_creationDate'] < group['release_year'], 'Team_compound'].sum() if not group[group['Team_creationDate'] < group['release_year']].empty else 0,
    #'Team_Review_Score_mean': group.loc[group['Team_creationDate'] < group['release_year'], 'Team_compound'].mean() if not group[group['Team_creationDate'] < group['release_year']].empty else 0,
    #'Team_Audience_Score_Mean': group.loc[group['Team_creationDate'] < group['release_year'], 'Team_audienceScore'].mean() if not group[group['Team_creationDate'] < group['release_year']].empty else 0,
    #'Team_Tomato_Meter_Mean': group.loc[group['Team_creationDate'] < group['release_year'], 'Team_tomatoMeter'].mean() if not group[group['Team_creationDate'] < group['release_year']].empty else 0,
    #'Team_Num_reviews': group.loc[group['Team_creationDate'] < group['release_year'], 'Team_reviewId'].nunique() if not group[group['Team_creationDate'] < group['release_year']].empty else 0,
    'Cast_Review_Score_sum': group.loc[group['Cast_creationDate'] < group['release_year'], 'Cast_compound'].sum() if not group[group['Cast_creationDate'] < group['release_year']].empty else 0,
    'Cast_Review_Score_mean': group.loc[group['Cast_creationDate'] < group['release_year'], 'Cast_compound'].mean() if not group[group['Cast_creationDate'] < group['release_year']].empty else 0,
    'Cast_Audience_Score_Mean': group.loc[group['Cast_creationDate'] < group['release_year'], 'Cast_audienceScore'].mean() if not group[group['Cast_creationDate'] < group['release_year']].empty else 0,
    'Cast_Tomato_Meter_Mean': group.loc[group['Cast_creationDate'] < group['release_year'], 'Cast_tomatoMeter'].mean() if not group[group['Cast_creationDate'] < group['release_year']].empty else 0,
    'Cast_Num_reviews': group.loc[group['Cast_creationDate'] < group['release_year'], 'Cast_reviewId'].nunique() if not group[group['Cast_creationDate'] < group['release_year']].empty else 0,
    **{col: group[col].iloc[0] for col in total.columns if col not in ['movie_id', 'Cast_compound', 'Cast_Review', 'Cast_reviewId']}
})).reset_index()

aggregated_data.head()

In [ ]:
aggregated_data.shape

In [ ]:
aggregated_data.columns

In [ ]:
aggregated_data['movie_id'].nunique()

In [ ]:

# Sentiment Score Sum vs Box Office Revenue
# Filter out box office outliers
Q1 = aggregated_data['box_office'].quantile(0.25)
Q3 = aggregated_data['box_office'].quantile(0.75)
IQR = Q3 - Q1
filtered_data = aggregated_data[(aggregated_data['box_office'] >= (Q1 - 1.5 * IQR)) & (aggregated_data['box_office'] <= (Q3 + 1.5 * IQR))]

plt.figure(figsize=(10, 6))
plt.scatter(filtered_data['Cast_Review_Score_sum'], filtered_data['box_office'], alpha=0.5)
plt.title('Review Sentiment Score Sum vs Box Office Numeric')
plt.xlabel('Review Sentiment Score Sum')
plt.ylabel('Box Office Numeric')
plt.grid(True)
plt.show()

# Sentiment Score Mean vs Box Office Revenue
plt.figure(figsize=(10, 6))
plt.scatter(filtered_data['Cast_Review_Score_mean'], filtered_data['box_office'], alpha=0.5)
plt.title('Review Sentiment Score Mean vs Box Office Numeric')
plt.xlabel('Review Sentiment Score Mean')
plt.ylabel('Box Office Numeric')
plt.grid(True)
plt.show()

In [ ]:
aggregated_data['Cast_Review_Score_sum'].describe()

In [ ]:
aggregated_data['Cast_Review_Score_mean'].describe()

In [ ]:
# Number of Reviews vs Box Office
plt.figure(figsize=(10, 6))
plt.scatter(filtered_data['Cast_Num_reviews'], filtered_data['box_office'], alpha=0.5)
plt.title('Number of Reviews vs Box Office')
plt.xlabel('Number of Reviews')
plt.ylabel('Box Office Numeric')
plt.grid(True)
plt.show()

In [ ]:
aggregated_data['Cast_Num_reviews'].describe()

In [ ]:
# Separate aggregated_data number of reviews into quantiles
aggregated_data['num_reviews_quantiles'] = pd.qcut(aggregated_data['Cast_Num_reviews'], q=5, labels=False)

# Check Interaction with Box Office
plt.figure(figsize=(10, 6))
aggregated_data.boxplot(column='box_office', by='num_reviews_quantiles')
plt.title('Box Office Numeric by Number of Reviews Quantiles')
plt.suptitle('')
plt.xlabel('Number of Reviews Quantiles')
plt.ylabel('Box Office Revenue')
plt.grid(True)
plt.show()

# Separate aggregated_data compound mean into quantiles
aggregated_data['compound_mean_quantiles'] = pd.qcut(aggregated_data['Review_Score_mean'], q=5, labels=False)

plt.figure(figsize=(10, 6))
aggregated_data.boxplot(column='box_office', by='compound_mean_quantiles')
plt.title('Box Office Numeric by Compound Mean Quantiles')
plt.suptitle('')
plt.xlabel('Compound Mean Quantiles')
plt.ylabel('Box Office Revenue')
plt.grid(True)
plt.show()


In [ ]:
### Movies that Lost Money
aggregated_data[aggregated_data['budget'] > aggregated_data['box_office']]

In [ ]:
### Cluster on these metrics
matched_metrics = aggregated_data[['movie_id', 'Cast_Review_Score_sum', 'Cast_Review_Score_mean', 'Cast_Audience_Score_Mean', 'Cast_Tomato_Meter_Mean', 'Cast_Num_reviews']]
matched_metrics.set_index('movie_id', inplace=True)

In [ ]:
matched_metrics

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Ensure no NA values in matched_metrics
matched_metrics = matched_metrics.dropna()

# Normalize the variables
scaler = StandardScaler()
matched_metrics_scaled = scaler.fit_transform(matched_metrics)

# Perform PCA
pca = PCA(n_components=2)
principal_components = pca.fit_transform(matched_metrics_scaled)

# Find the optimal number of clusters
inertia_values = []
silhouette_scores = []
K = range(2, 11)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(principal_components)
    inertia_values.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(principal_components, kmeans.labels_))

# Plot inertia 
plt.figure(figsize=(10, 6))
plt.plot(K, inertia_values, marker='o')
plt.title('Inertia for Different Numbers of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()

# Plot silhouette scores 
plt.figure(figsize=(10, 6))
plt.plot(K, silhouette_scores, marker='x')
plt.title('Silhouette Score for Different Numbers of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.grid(True)
plt.show()

In [ ]:
# Perform KMeans clustering 
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(principal_components)
labels = kmeans.labels_

# Add the cluster labels to the matched_metrics 
matched_metrics['Cluster'] = labels

# Define cluster labels
#cluster_labels = {3: 'Blockbuster Ensemble', 2: 'Critically Acclaimed', 0: 'Steady Performers', 1: 'Low Impact Ensemble'}
#matched_metrics['Cluster Label'] = matched_metrics['Cluster'].map(cluster_labels)

# Plot clusters with labels
plt.figure(figsize=(10, 6))
scatter = plt.scatter(principal_components[:, 0], principal_components[:, 1], c=matched_metrics['Cluster'], cmap='viridis')
plt.title('Clusters of Matched Metrics')
plt.xlabel('First Metric')
plt.ylabel('Second Metric')
plt.grid(True)
plt.colorbar(scatter, label='Cluster')
# Add legend with cluster labels
handles, _ = scatter.legend_elements()
#legend_labels = [cluster_labels[i] for i in range(len(handles))]
#plt.legend(handles, legend_labels, title="Cluster Labels")
plt.show()

In [ ]:
# For each cluster, get description of principal components
numeric_columns = matched_metrics.select_dtypes(include='number').columns
cluster_descriptions = matched_metrics.groupby('Cluster')[numeric_columns].mean()
cluster_descriptions

In [ ]:
matched_metrics['Cluster'].value_counts()

In [ ]:
# Define cluster labels
cluster_labels = {0: 'Emerging Ensemble', 1: 'Unknown Cast', 2: 'Star Studded Cast', 3: 'Steady Performers'}
matched_metrics['Cluster Label'] = matched_metrics['Cluster'].map(cluster_labels)

# Display the first few rows to verify the new column
matched_metrics.head()


In [ ]:
plt.figure(figsize=(10, 6))
scatter = plt.scatter(principal_components[:, 0], principal_components[:, 1], c=matched_metrics['Cluster'], cmap='viridis')
plt.title('Clusters of Matched Metrics')
plt.xlabel('First Metric')
plt.ylabel('Second Metric')
plt.grid(True)
plt.colorbar(scatter, label='Cluster')
# Add legend with cluster labels
handles, _ = scatter.legend_elements()
legend_labels = [cluster_labels[i] for i in range(len(handles))]
plt.legend(handles, legend_labels, title="Cluster Labels")
plt.show()

In [ ]:
# Pull the Cluster Label and merge back to aggregated_data
aggregated_data = aggregated_data.merge(matched_metrics[['Cluster Label']], left_on='movie_id', right_index=True, how='left')
aggregated_data.head()

In [ ]:
# Change the name of the column 'Cluster Label' to 'Cast_Cluster_Label'
aggregated_data.rename(columns={'Cluster Label': 'Cast_Cluster_Label'}, inplace=True)
aggregated_data.head()


In [ ]:
team = pd.read_csv('team_reviews.csv')
team1 = team.drop_duplicates()

In [ ]:
team1.head()

In [ ]:
team1['Sentiment'] = team1['Review'].apply(lambda review: analyzer.polarity_scores(review))
team1['compound'] = team1['Sentiment'].apply(lambda sentiment: sentiment['compound'])

In [ ]:
exploded_aggregated_data = aggregated_data.explode('Team')

In [ ]:
exploded_aggregated_data.drop(columns=['Name'], inplace=True)

In [ ]:
total = exploded_aggregated_data.merge(team1,left_on='Team', right_on='Name', how='left')

In [ ]:
total.head()

In [ ]:
total.shape

In [ ]:
total.rename(columns={
    'audienceScore': 'Team_audienceScore',
    'tomatoMeter': 'Team_tomatoMeter',
    'Sentiment': 'Team_Sentiment',
    'compound': 'Team_compound',
    'Review': 'Team_Review',
    'reviewId': 'Team_reviewId',
    'creationDate': 'Team_creationDate'
}, inplace=True)
total.head()

In [ ]:
aggregated_data_team_cast = total.groupby('movie_id').apply(lambda group: pd.Series({
    'Team_Review_Score_sum': group.loc[group['Team_creationDate'] < group['release_year'], 'Team_compound'].sum() if not group[group['Team_creationDate'] < group['release_year']].empty else 0,
    'Team_Review_Score_mean': group.loc[group['Team_creationDate'] < group['release_year'], 'Team_compound'].mean() if not group[group['Team_creationDate'] < group['release_year']].empty else 0,
    'Team_Audience_Score_Mean': group.loc[group['Team_creationDate'] < group['release_year'], 'Team_audienceScore'].mean() if not group[group['Team_creationDate'] < group['release_year']].empty else 0,
    'Team_Tomato_Meter_Mean': group.loc[group['Team_creationDate'] < group['release_year'], 'Team_tomatoMeter'].mean() if not group[group['Team_creationDate'] < group['release_year']].empty else 0,
    'Team_Num_reviews': group.loc[group['Team_creationDate'] < group['release_year'], 'Team_reviewId'].nunique() if not group[group['Team_creationDate'] < group['release_year']].empty else 0,
    #'Cast_Review_Score_sum': group.loc[group['Cast_creationDate'] < group['release_year'], 'Cast_compound'].sum() if not group[group['Cast_creationDate'] < group['release_year']].empty else 0,
    #'Cast_Review_Score_mean': group.loc[group['Cast_creationDate'] < group['release_year'], 'Cast_compound'].mean() if not group[group['Cast_creationDate'] < group['release_year']].empty else 0,
    #'Cast_Audience_Score_Mean': group.loc[group['Cast_creationDate'] < group['release_year'], 'Cast_audienceScore'].mean() if not group[group['Cast_creationDate'] < group['release_year']].empty else 0,
    #'Cast_Tomato_Meter_Mean': group.loc[group['Cast_creationDate'] < group['release_year'], 'Cast_tomatoMeter'].mean() if not group[group['Cast_creationDate'] < group['release_year']].empty else 0,
    #'Cast_Num_reviews': group.loc[group['Cast_creationDate'] < group['release_year'], 'Cast_reviewId'].nunique() if not group[group['Cast_creationDate'] < group['release_year']].empty else 0,
    **{col: group[col].iloc[0] for col in total.columns if col not in ['movie_id', 'Team_compound', 'Team_Review', 'Team_reviewId']}
})).reset_index()

aggregated_data_team_cast.head()

In [ ]:
matched_metrics_team_cast = aggregated_data_team_cast[['movie_id', 'Team_Review_Score_sum', 'Team_Review_Score_mean', 'Team_Audience_Score_Mean', 'Team_Tomato_Meter_Mean', 'Team_Num_reviews']]
matched_metrics_team_cast.set_index('movie_id', inplace=True)

In [ ]:
matched_metrics_team_cast.head()

In [ ]:
matched_metrics_team_cast= matched_metrics_team_cast.dropna()

# Normalize the variables
scaler = StandardScaler()
matched_metrics_team_cast_scaled = scaler.fit_transform(matched_metrics_team_cast)

# Perform PCA
pca = PCA(n_components=2)
principal_components = pca.fit_transform(matched_metrics_team_cast_scaled)

# Find the optimal number of clusters
inertia_values = []
silhouette_scores = []
K = range(2, 11)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(principal_components)
    inertia_values.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(principal_components, kmeans.labels_))

# Plot inertia 
plt.figure(figsize=(10, 6))
plt.plot(K, inertia_values, marker='o')
plt.title('Inertia for Different Numbers of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()

# Plot silhouette scores 
plt.figure(figsize=(10, 6))
plt.plot(K, silhouette_scores, marker='x')
plt.title('Silhouette Score for Different Numbers of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.grid(True)
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(principal_components)
labels = kmeans.labels_

# Add the cluster labels to the matched_metrics 
matched_metrics_team_cast['Cluster'] = labels

# Define cluster labels
#cluster_labels = {3: 'Blockbuster Ensemble', 2: 'Critically Acclaimed', 0: 'Steady Performers', 1: 'Low Impact Ensemble'}
#matched_metrics['Cluster Label'] = matched_metrics['Cluster'].map(cluster_labels)

# Plot clusters with labels
plt.figure(figsize=(10, 6))
scatter = plt.scatter(principal_components[:, 0], principal_components[:, 1], c=matched_metrics_team_cast['Cluster'], cmap='viridis')
plt.title('Clusters of Matched Metrics')
plt.xlabel('First Metric')
plt.ylabel('Second Metric')
plt.grid(True)
plt.colorbar(scatter, label='Cluster')
# Add legend with cluster labels
handles, _ = scatter.legend_elements()
#legend_labels = [cluster_labels[i] for i in range(len(handles))]
#plt.legend(handles, legend_labels, title="Cluster Labels")
plt.show()

In [ ]:
numeric_columns = matched_metrics_team_cast.select_dtypes(include='number').columns
cluster_descriptions = matched_metrics_team_cast.groupby('Cluster')[numeric_columns].mean()
cluster_descriptions

In [ ]:
matched_metrics_team_cast['Cluster'].value_counts()

In [ ]:
# Define cluster labels
cluster_labels = {0: 'Unknown Director / Writer', 1: 'Emerging Director / Writer', 2: 'Blockbuster Team', 3: 'Established Team'}
matched_metrics_team_cast['Cluster Label'] = matched_metrics_team_cast['Cluster'].map(cluster_labels)

# Display the first few rows to verify the cluster labels
matched_metrics_team_cast.head()


In [ ]:
plt.figure(figsize=(10, 6))
scatter = plt.scatter(principal_components[:, 0], principal_components[:, 1], c=matched_metrics_team_cast['Cluster'], cmap='viridis')
plt.title('Clusters of Matched Metrics')
plt.xlabel('First Metric')
plt.ylabel('Second Metric')
plt.grid(True)
plt.colorbar(scatter, label='Cluster')
# Add legend with cluster labels
handles, _ = scatter.legend_elements()
legend_labels = [cluster_labels[i] for i in range(len(handles))]
plt.legend(handles, legend_labels, title="Cluster Labels")
plt.show()

In [ ]:
aggregated_data_team_cast = aggregated_data_team_cast.merge(matched_metrics_team_cast[['Cluster Label']], left_on='movie_id', right_index=True, how='left')
aggregated_data_team_cast.head()

In [ ]:
# Drop the 'Team_Sentiment' column
aggregated_data_team_cast = aggregated_data_team_cast.drop(columns=['Team_Sentiment'])

# Rename 'Cluster Label' to 'Team_Cluster Label'
aggregated_data_team_cast = aggregated_data_team_cast.rename(columns={'Cluster Label': 'Team_Cluster Label'})

# Display the first few rows to verify the changes
aggregated_data_team_cast.head()


In [ ]:
aggregated_data_team_cast.shape

In [ ]:
### FINAL DATASET
aggregated_data_team_cast.head()

In [ ]:
aggregated_data_team_cast.columns

In [ ]:
aggregated_data_team_cast.to_csv('final_dataset_cam.csv', index=False)